## Custom Segmentation Algorithm Profling

Notebook to profile segmentation implementation ([spheroid_cytometer.py](spheroid_cytometer.py)) for performance deficiencies.

In [1]:
%matplotlib inline
%run spheroid_cytometer.py
import os
import os.path as osp
import numpy as np
import matplotlib.pyplot as plt
from cytokit.ops import cytometry as cytometer_op
from cytokit.ops import tile_generator
from cytokit import math as ck_math
from cytokit import config as ck_config

In [2]:
exp_name = '20190215-mc38-dmso-control'
exp_dir = 'XY03'
data_dir = osp.join(os.environ['CYTOKIT_DATA_DIR'], 'spheroid', exp_name, 'gdoc_samp', exp_dir, 'raw')
config_dir = osp.join(os.environ['CYTOKIT_ANALYSIS_REPO_DIR'], 'config', 'experiment', 'spheroid', exp_name)
config = ck_config.load(config_dir)
os.environ['CYTOKIT_PATH_FORMATS'] = 'get_default_path_formats("1_' + exp_dir + '_{tile:05d}_Z{z:03d}_CH{channel:d}.tif")'
config.register_environment()

In [3]:
tile_gen = tile_generator.CytokitTileGenerator(config, data_dir, 0, 4)

In [4]:
tile = tile_gen.run()
tile.shape

(1, 26, 3, 1440, 1920)

In [5]:
op = cytometer_op.Cytometry2D(config)

In [6]:
op.initialize()

In [7]:
# !pip install line_profiler
%load_ext line_profiler

In [12]:
img_seg = op.cytometer.segment(tile[0, :, 0])
img_seg.shape

(26, 6, 1440, 1920)

In [13]:
stats = op.cytometer.quantify(tile, img_seg, channel_names=config.channel_names, morphology_features=True, nucleus_intensity=['mean', 'sum'])

In [14]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1014 entries, 0 to 1013
Data columns (total 33 columns):
id                            1014 non-null int64
x                             1014 non-null float64
y                             1014 non-null float64
z                             1014 non-null int64
cm:size                       1014 non-null int64
cm:diameter                   1014 non-null float64
cm:perimeter                  1014 non-null float64
cm:solidity                   1014 non-null float64
nm:size                       1014 non-null int64
nm:diameter                   1014 non-null float64
nm:perimeter                  1014 non-null float64
nm:solidity                   1014 non-null float64
ci:BF:mean                    1014 non-null float64
ci:LIVE:mean                  1014 non-null float64
ci:DEAD:mean                  1014 non-null float64
ni:BF:mean                    1014 non-null float64
ni:LIVE:mean                  1014 non-null float64
ni:DEAD:mean     

In [21]:
%lprun -f op.cytometer.segment img_seg = op.cytometer.segment(tile[0, :, 0])

Timer unit: 1e-06 s

Total time: 9.36233 s
File: /lab/repos/cell-image-analysis/analysis/experiments/spheroid/20190215-mc38-dmso-control/spheroid_cytometer.py
Function: segment at line 21

Line #      Hits         Time  Per Hit   % Time  Line Contents
    21                                               def segment(self, img, **kwargs):
    22         1         11.0     11.0      0.0          assert img.ndim == 3, 'Expecting 3D image, got shape %s' % img.shape
    23                                                   
    24                                                   # Run max-z projection to create single 2D image (but record initial number of z planes)
    25         1          6.0      6.0      0.0          nz = img.shape[0]
    26         1     137033.0 137033.0      1.5          img = img.max(axis=0)
    27                                                   
    28                                                   # Verify 8 or 16 bit type before forcing to 8 bit (for median 